# Mandelbrot With Blosc2 DSL vs Numba

This notebook builds a Mandelbrot kernel with `@blosc2.dsl_kernel`, following the structure in `../miniexpr/examples/13_mandelbrot_dsl.c`:
- state variables (`zr`, `zi`, `escape_iter`)
- a `for` loop over iterations
- conditional updates with `where`
- early `break` when all points escaped

Then it times the same computation implemented with Numba and plots both outputs.

In [ ]:
import time

import matplotlib.pyplot as plt
import numpy as np
from numba import njit, prange

import blosc2

In [ ]:
# Problem size and Mandelbrot domain
WIDTH = 1200
HEIGHT = 800
MAX_ITER = 200
X_MIN, X_MAX = -2.0, 0.6
Y_MIN, Y_MAX = -1.1, 1.1
DTYPE = np.float32

x = np.linspace(X_MIN, X_MAX, WIDTH, dtype=DTYPE)
y = np.linspace(Y_MIN, Y_MAX, HEIGHT, dtype=DTYPE)
cr_np, ci_np = np.meshgrid(x, y)

# Keep compression overhead low for the timing comparison
cparams_fast = blosc2.CParams(codec=blosc2.Codec.LZ4, clevel=1)
cr_b2 = blosc2.asarray(cr_np, cparams=cparams_fast)
ci_b2 = blosc2.asarray(ci_np, cparams=cparams_fast)

print(f"grid: {cr_np.shape}, dtype: {cr_np.dtype}")

In [ ]:
@blosc2.dsl_kernel
def mandelbrot_dsl(cr, ci, max_iter):
    # max_iter is passed per-call through lazyudf inputs
    zr = 0.0
    zi = 0.0
    escape_iter = float(max_iter)
    for it in range(max_iter):
        zr2 = zr * zr
        zi2 = zi * zi
        mag2 = zr2 + zi2

        active = escape_iter == float(max_iter)
        just_escaped = (mag2 > 4.0) & active
        escape_iter = np.where(just_escaped, it, escape_iter)

        active = escape_iter == float(max_iter)
        if np.all(active == 0):
            break

        zr_new = zr2 - zi2 + cr
        zi_new = 2.0 * zr * zi + ci
        zr = np.where(active, zr_new, zr)
        zi = np.where(active, zi_new, zi)

    return escape_iter


if mandelbrot_dsl.dsl_source is None:
    raise RuntimeError("DSL extraction failed. Re-run this cell in a file-backed notebook session.")

print(mandelbrot_dsl.dsl_source)

In [ ]:
@njit(parallel=True, fastmath=False)
def mandelbrot_numba(cr, ci, max_iter):
    h, w = cr.shape
    out = np.empty((h, w), dtype=np.float32)
    for iy in prange(h):
        for ix in range(w):
            zr = np.float32(0.0)
            zi = np.float32(0.0)
            escape_iter = np.float32(max_iter)
            c_re = cr[iy, ix]
            c_im = ci[iy, ix]
            for it in range(max_iter):
                zr2 = zr * zr
                zi2 = zi * zi
                if zr2 + zi2 > np.float32(4.0):
                    escape_iter = np.float32(it)
                    break
                zr_new = zr2 - zi2 + c_re
                zi_new = np.float32(2.0) * zr * zi + c_im
                zr = zr_new
                zi = zi_new
            out[iy, ix] = escape_iter
    return out

In [ ]:
def best_time(func, repeats=3, warmup=1):
    for _ in range(warmup):
        func()
    best = float("inf")
    best_out = None
    for _ in range(repeats):
        t0 = time.perf_counter()
        out = func()
        dt = time.perf_counter() - t0
        if dt < best:
            best = dt
            best_out = out
    return best, best_out


def run_dsl():
    lazy = blosc2.lazyudf(mandelbrot_dsl, (cr_b2, ci_b2, MAX_ITER), dtype=np.float32, cparams=cparams_fast)
    return lazy.compute()[...]


def run_numba():
    return mandelbrot_numba(cr_np, ci_np, MAX_ITER)


# Measure first iteration (includes one-time overhead, especially Numba JIT compile)
t0 = time.perf_counter()
_ = run_dsl()
t_dsl_first = time.perf_counter() - t0

t0 = time.perf_counter()
_ = run_numba()
t_numba_first = time.perf_counter() - t0

t_dsl, img_dsl = best_time(run_dsl, repeats=3, warmup=1)
t_numba, img_numba = best_time(run_numba, repeats=5, warmup=1)
max_abs_diff = float(np.max(np.abs(img_dsl - img_numba)))

print("First iteration timings (one-time overhead included):")
print(f"DSL first run:    {t_dsl_first:.6f} s")
print(f"Numba first run:  {t_numba_first:.6f} s")
print(f"DSL / Numba:      {t_dsl_first / t_numba_first:.2f}x")

print("\nBest-time stats:")
print(f"DSL time (best):   {t_dsl:.6f} s")
print(f"Numba time (best): {t_numba:.6f} s")
print(f"DSL / Numba:       {t_dsl / t_numba:.2f}x")
print(f"max |dsl-numba|:   {max_abs_diff:.6f}")

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(13, 5), constrained_layout=True)

im0 = ax[0].imshow(
    img_dsl,
    cmap="magma",
    extent=(X_MIN, X_MAX, Y_MIN, Y_MAX),
    origin="lower",
)
ax[0].set_title("Mandelbrot (Blosc2 DSL)")
ax[0].set_xlabel("Re(c)")
ax[0].set_ylabel("Im(c)")
fig.colorbar(im0, ax=ax[0], shrink=0.82, label="Escape iteration")

im1 = ax[1].imshow(
    img_numba,
    cmap="magma",
    extent=(X_MIN, X_MAX, Y_MIN, Y_MAX),
    origin="lower",
)
ax[1].set_title("Mandelbrot (Numba)")
ax[1].set_xlabel("Re(c)")
ax[1].set_ylabel("Im(c)")
fig.colorbar(im1, ax=ax[1], shrink=0.82, label="Escape iteration")

plt.show()

In [ ]:
labels = ["Blosc2 DSL", "Numba"]
first_times = [t_dsl_first, t_numba_first]
best_times = [t_dsl, t_numba]

x = np.arange(len(labels))
width = 0.36

fig, ax = plt.subplots(figsize=(8, 5), constrained_layout=True)
ax.bar(x - width / 2, first_times, width, label="First run", color="#4C78A8")
ax.bar(x + width / 2, best_times, width, label="Best run", color="#F58518")

ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.set_ylabel("Time (seconds)")
ax.set_title("Mandelbrot Timings: First vs Best")
ax.legend()

for i, t in enumerate(first_times):
    ax.text(i - width / 2, t, f"{t:.3f}s", ha="center", va="bottom")
for i, t in enumerate(best_times):
    ax.text(i + width / 2, t, f"{t:.3f}s", ha="center", va="bottom")

plt.show()